In [1]:
import os
import gc
import sys
import numpy as np
import json
from pympler import tracker
from datetime import datetime
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from data_helpers import sample_train_test
from tempConv import tempConvDecoder
# from other import get_sha

Using TensorFlow backend.


In [2]:
tr = tracker.SummaryTracker()

conf_path = os.path.abspath('datasets/GRatX/ridge_config.json') # pass path to config here as arg
rat_path = os.path.dirname(conf_path)
conf = json.load(open(conf_path, 'r'))
# conf['git_version'] = get_sha()

data_keys = [folder for folder in os.listdir(rat_path)]
folders = [os.path.join(rat_path,folder) for folder in os.listdir(rat_path)]
folders = filter(lambda x: os.path.isdir(x), folders)

X_fname = conf['config']['neural_data']
y_fname = conf['config']['head_data']
dataset_paths = []
            
## produce dataset_paths, format: [[path/to/X1, path/to/y1], [path/to/X2, path/to/y2]..]
## example of collecting some datasets into a list of tuples
for folder in folders:
    data_file_list = os.listdir(folder)
    if True in [conf['config']['condition'] in fil for fil in data_file_list]:
        if True in [conf['config']['neural_data'] in fil for fil in data_file_list]:
            dataset_paths.append([
                folder+'/'+data_file_list[data_file_list.index(X_fname)],
                folder+'/'+data_file_list[data_file_list.index(y_fname)]
            ])

dataset_paths = np.array(dataset_paths)
dataset_paths

array([[ '/home/joeldapello/Code/v1-decoder/datasets/GRatX/636440035877005948/mua_firing_rates_100hz.hdf5',
        '/home/joeldapello/Code/v1-decoder/datasets/GRatX/636440035877005948/all_head_data_100hz.hdf5'],
       [ '/home/joeldapello/Code/v1-decoder/datasets/GRatX/636426429249996956/mua_firing_rates_100hz.hdf5',
        '/home/joeldapello/Code/v1-decoder/datasets/GRatX/636426429249996956/all_head_data_100hz.hdf5'],
       [ '/home/joeldapello/Code/v1-decoder/datasets/GRatX/636432491422312001/mua_firing_rates_100hz.hdf5',
        '/home/joeldapello/Code/v1-decoder/datasets/GRatX/636432491422312001/all_head_data_100hz.hdf5']], 
      dtype='<U95')

In [6]:
conf_nn = conf['nn_params']
def make_ridgeCV_model():    
    print('********************************** Making RidgeCV Model **********************************')
    #Declare model
    model = Ridge(alpha=0.1, normalize=True,fit_intercept=True)
  
    return model

stats = []

# kfold split dataset_paths, n-1 to train, 1 to test
kf = KFold(n_splits=len(dataset_paths))
for train_idx, test_idx in kf.split(dataset_paths):
    train_paths = dataset_paths[train_idx]
    test_paths = dataset_paths[test_idx]
    
    X_train, y_train, X_test, y_test = [None, None, None, None] # delete residual data first
    X_train, y_train, X_test, y_test = sample_train_test(train_paths, test_paths, conf_nn)    
    X_train = X_train.reshape(X_train.shape[0],(X_train.shape[1]*X_train.shape[2]))
    X_test = X_test.reshape(X_test.shape[0],(X_test.shape[1]*X_test.shape[2]))
    
    model = make_ridgeCV_model()
    model.fit(X_test, y_test)
    stats.append(model.score(X_train,y_train))
    stats.append(model.score(X_test,y_test))

print(stats)

sample train loaded: [[ '/home/joeldapello/Code/v1-decoder/datasets/GRatX/636426429249996956/mua_firing_rates_100hz.hdf5'
  '/home/joeldapello/Code/v1-decoder/datasets/GRatX/636426429249996956/all_head_data_100hz.hdf5']
 [ '/home/joeldapello/Code/v1-decoder/datasets/GRatX/636432491422312001/mua_firing_rates_100hz.hdf5'
  '/home/joeldapello/Code/v1-decoder/datasets/GRatX/636432491422312001/all_head_data_100hz.hdf5']]
sample test loaded: [[ '/home/joeldapello/Code/v1-decoder/datasets/GRatX/636440035877005948/mua_firing_rates_100hz.hdf5'
  '/home/joeldapello/Code/v1-decoder/datasets/GRatX/636440035877005948/all_head_data_100hz.hdf5']]
********************************** Making RidgeCV Model **********************************
sample train loaded: [[ '/home/joeldapello/Code/v1-decoder/datasets/GRatX/636440035877005948/mua_firing_rates_100hz.hdf5'
  '/home/joeldapello/Code/v1-decoder/datasets/GRatX/636440035877005948/all_head_data_100hz.hdf5']
 [ '/home/joeldapello/Code/v1-decoder/datasets/GR

In [4]:
model = make_ridgeCV_model()
model.fit(X_test, y_test)

********************************** Making RidgeCV Model **********************************


Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=True, random_state=None, solver='auto', tol=0.001)

In [5]:
model.score(X_test,y_test)

0.089436252348787715

In [ ]:
from scipy import stats,signal

def filter(ephys,freq_range,filt_order = 4,filt_type='bandpass',fs=10.):
    
    # design Elliptic filter:

    [b,a] = signal.butter(filt_order,[freq/(fs/2) for freq in freq_range],btype=filt_type)
    
    filtered_trace = signal.filtfilt(b,a,ephys,axis=0)
    return filtered_trace



In [7]:
X_train.shape

(1116350, 960)

In [23]:
X_train = X_train.reshape(X_train.shape[0],(X_train.shape[1]*X_train.shape[2]))
X_test = X_test.reshape(X_test.shape[0],(X_test.shape[1]*X_test.shape[2]))
X_train.shape, X_test.shape

((1116350, 960), (560861, 960))

In [8]:
model.score(X_train,y_train)

0.02366116208214919

In [33]:
from sklearn.linear_model import Ridge

def make_ridgeCV_model():    
    print('********************************** Making RidgeCV Model **********************************')
    #Declare model
    model = Ridge(alpha=0.1, normalize=True,fit_intercept=True)
  
    return model

model = make_ridgeCV_model()
model.fit(X_train, y_train)
model.score(X_test,y_test)

********************************** Making RidgeCV Model **********************************


-0.022021158769203186

In [29]:
y_hat = model.predict(X_test)
# model.score(y_hat,y_test)
y_hat.shape

(560861, 1)

In [30]:
y_test.shape

(560861, 1)

In [32]:
model.score(X_test,y_test)

-0.022021158769203186

In [9]:
# kfold split dataset_paths, n-1 to train, 1 to test
kf = KFold(n_splits=len(dataset_paths))
for train_idx, test_idx in kf.split(dataset_paths):
    train_paths = dataset_paths[train_idx]
    test_paths = dataset_paths[test_idx]

    print("train on:\n %s\n test on:\n %s\n" % (train_paths, test_paths))

    conf_nn = conf['nn_params']
    conf_nn['run_id'] = str(datetime.now().timestamp())
    conf_nn['train_paths'] = train_paths.tolist()
    conf_nn['test_paths'] = test_paths.tolist()
    save_path = os.path.join(rat_path, conf['config']['experiment'], conf_nn['run_id'])
    if not os.path.exists(save_path):
        print('create save directory: ', save_path)
        os.makedirs(save_path)
    conf_nn['save_path'] = save_path
    
    # import pdb; pdb.set_trace()   # uncomment for interactive debugging 
    
    # collect statistics from each epoch
    stats = []
    
    ## train for number of epochs, resampling training datasets at each new epoch
    for epoch in range(conf_nn['eps']):
        #### train model and test model for each Kfold
        # sample first round of data:
        X_train, y_train, X_test, y_test = [None, None, None, None] # delete residual data first
        X_train, y_train, X_test, y_test = sample_train_test(train_paths, test_paths, conf_nn)

        if epoch == 0:
            conf_nn['input_shape'] = X_train.shape
            conf_nn['output_shape'] = y_train.shape

            ## define model on first epoch
            TCD = tempConvDecoder(**conf_nn)

        print('epoch: %s/%s' % (epoch, conf_nn['eps']))

        TCD.model.fit(
            X_train,
            y_train,
            epochs=1, # ignore, we control epochs with for loop  
            batch_size=conf_nn['bs']
        )
        
        # ugly plot results if final epoch
        if epoch+1 == int(conf_nn['eps']):
            R2s,rs = TCD.determine_fit(X_test, y_test, save_result=True)
        else:
            R2s,rs = TCD.determine_fit(X_test, y_test, save_result=False)

        stats.append([R2s, rs])
    
        print("R2: %s\n r: %s" % (R2s, rs))

        gc.collect()
        tr.print_diff()
    
    conf_nn['stats'] = stats

    # save stats and settings after last epoch
    with open(os.path.join(save_path, 'trial_info.json'), 'w') as outfile:
        json.dump(conf, outfile)
    
    print(stats)

array([[ '/home/joeldapello/Code/v1-decoder/datasets/GRatX/636440035877005948/mua_firing_rates_100hz.hdf5',
        '/home/joeldapello/Code/v1-decoder/datasets/GRatX/636440035877005948/all_head_data_100hz.hdf5'],
       [ '/home/joeldapello/Code/v1-decoder/datasets/GRatX/636426429249996956/mua_firing_rates_100hz.hdf5',
        '/home/joeldapello/Code/v1-decoder/datasets/GRatX/636426429249996956/all_head_data_100hz.hdf5']], 
      dtype='<U95')